# BCI_005_Matriz_Comercial_B1_Factoring

## Información del Notebook

### Encabezado
**************************************************************************
* Nombre: 
* Ruta: 
* Autor: Esteban Gándara
* Ing. Rafael Montecinos - rafael.montecinost@bci.cl
* Fecha: 19/05/2025
* Descripción: 
* Documentación: 
***************************************************************************

### Mantenciones
**************************************************************************
#### Mantención Nro: 
* Autor: <Nombre Autor> (<Empresa del Autor (Bci/Otra)>) - Ing. SW BCI: <Nombre Ing. SW BCI>
* Fecha: <dd/mm/yyyy> 
* Descripción: <Descripción de la mantención>      
************************************************

### Tablas Entrada y Salida
**************************************************************************
#### Tablas Entrada: 
* 
* 
***************************************************************************
#### Tablas Salida: 
* 

## Captura de Variables

In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.text("fechaProcesoW","","01 Fecha Proceso :")
dbutils.widgets.text("platinum_temp_dbW","","02 platinum temp db:")
dbutils.widgets.text("db_location_platinum_tempW","","03 Location DB Platinum Temp:")

### Asignar Objeto a Lectura de Widgets y Variables

In [0]:
fechaProcesoX = dbutils.widgets.get("fechaProcesoW")
spark.conf.set("bci.fechaProcesoX", fechaProcesoX)

platinum_temp_dbX = dbutils.widgets.get("platinum_temp_dbW")
spark.conf.set("bci.platinum_temp_dbX", platinum_temp_dbX)

db_location_platinum_tempX = dbutils.widgets.get("db_location_platinum_tempW")
spark.conf.set("bci.db_location_platinum_tempX", db_location_platinum_tempX)

print("*****Parámetros*****")
print("fechaProcesoX: " + fechaProcesoX)
print("platinum_temp_dbX: " + platinum_temp_dbX)
print("db_location_platinum_tempX: " + db_location_platinum_tempX)

*****Parámetros*****
fechaProcesoX: 20250530
platinum_temp_dbX: dsr_plt_bcitemp_db
db_location_platinum_tempX: abfss://platinum@bcirg2dlssbx.dfs.core.windows.net/plt_normativo_db/prv_hip/


### Librerías

In [0]:
import json
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import relativedelta

# Obtener fecha actual
current_date_str = datetime.now().strftime("%Y-%m-%d")
current_date = datetime.strptime(current_date_str, "%Y-%m-%d")
formatted_date = str(current_date.strftime("%Y%m%d"))

###Asigan Variables de fecha

In [0]:
ano = str(fechaProcesoX)[:4]
mes = str(fechaProcesoX)[4:][:2]
dia = str(fechaProcesoX)[6:][:2]
fechanormativo = str(ano+'-'+mes+'-'+dia)
fechacinta = str(dia+'-'+mes+'-'+ano)
anomes = str(ano+mes)
anomesdia = str(ano+mes+dia)
anomesdia2 = str(ano+'/'+mes+'/'+dia)
anomesdia3 = str(ano+'-'+mes+'-'+'01')


print("fecha_Formato1: " + ano)
print("fecha_Formato2: " + mes)
print("fecha_Formato3: " + dia)
print("fecha_Formato4: " + fechanormativo)
print("fecha_Formato5: " + fechacinta)
print("fecha_Formato6: " + anomes)
print("fecha_Formato7: " + anomesdia)
print("fecha_formato8: " + anomesdia2)
print("fecha_formato9: " + anomesdia3)

fecha_Formato1: 2025
fecha_Formato2: 05
fecha_Formato3: 30
fecha_Formato4: 2025-05-30
fecha_Formato5: 30-05-2025
fecha_Formato6: 202505
fecha_Formato7: 20250530
fecha_formato8: 2025/05/30
fecha_formato9: 2025-05-01


## Funciones

In [0]:
%run "./Funciones"

Fecha vacía

Fecha futura

## Validaciones

### Fecha vacía

In [0]:
fecha_vacia(fechaProcesoX)

### Fecha futura

In [0]:
fecha_futura(fechaProcesoX)

## Inicio de Lógica

### Creación de tablas temporales que se eliminarán al final de la ejecución

**************************************************************************

#### Prov_Gr_Com_Fact_B1_Result_0719

In [0]:
drop_tmp_008_Tmp_Prov_Gr_Com_Fact_B1_Result_0719 = """ DROP TABLE IF EXISTS """ + platinum_temp_dbX + """.Tmp_Prov_Gr_Com_Fact_B1_Result_0719 """

In [0]:
sql_safe(drop_tmp_008_Tmp_Prov_Gr_Com_Fact_B1_Result_0719)

sql_safe: query ->  DROP TABLE IF EXISTS dsr_plt_bcitemp_db.Tmp_Prov_Gr_Com_Fact_B1_Result_0719 


DataFrame[]

In [0]:
dbutils.fs.rm(db_location_platinum_tempX+"Tmp_Prov_Gr_Com_Fact_B1_Result_0719", True)

True

In [0]:
create_tmp_008_Tmp_Prov_Gr_Com_Fact_B1_Result_0719 = """ CREATE TABLE IF NOT EXISTS """ + platinum_temp_dbX + """.Tmp_Prov_Gr_Com_Fact_B1_Result_0719 
USING DELTA
PARTITIONED BY (periodo)
LOCATION '""" + db_location_platinum_tempX + """Tmp_Prov_Gr_Com_Fact_B1_Result_0719'
AS (
select 
Cod_niid AS ope_num
--,dpf_fec_proceso
,cli_idc as rut
,cli_vrt as dv
--,cli_rzn_soc
,ddr_idc as rut_deudor_fact
,ddr_vrt as dv_deudor_fact
--,ddr_rzn_soc
--,pdt_des_cra
,doc_num_documento
--,doc_num_cuota
,Deuda_TotalIFRS
--,dod_cod_cobranza
--,cod_aec
--,dpf_ind_responsabilidad
--,dpf_fec_vencimiento
,Clasificacion as seg_n2
--,dod_ind_cartera
--,Clasificacion_Cliente
,Clasificacion_Deudor as Clasificacion_Deudor_fact
,Dias_Mora
,CASE WHEN Dias_Mora=0 and Colo_EsCardetIrr=0 then "0"
WHEN Dias_Mora>=1 and Dias_Mora<=29 and Colo_EsCardetIrr=0 then "1-29"
WHEN Dias_Mora>=30 and Dias_Mora<=59 and Colo_EsCardetIrr=0 then "30-59"
WHEN Dias_Mora>=60 and Dias_Mora<=89 and Colo_EsCardetIrr=0 then "60-89"
else "Incumpl" END AS Tramo_Mora
,doc_num_operacion as doc_num_operacion_fact
,doc_id_documento as doc_id_documento_fact
--,doc_ind_fin_mes
--,dpf_pct_pi
--,dpf_pct_pdi
--,dpf_mto_pe
--,cli_rut
,Responsabilidad_CedenteFactoring
,Tipo_Prestamo
--,Periodo_Id
,Ind_Castigo
,Colo_EsCardetIrr
--,Calif_Deudor_Permitida
,PI
,PDI
,PI_A
,PDI_A
--,Mitigador
,PI*PDI*Deuda_TotalIFRS AS PROV_d00
,"BCI" AS Banco
,Tipo_Prestamo AS Prestamo
,CASE WHEN trim(Tipo_Prestamo)="ESTUDIANTIL" then "EST"
WHEN trim(Tipo_Prestamo)="LEASING" then "LEA"
else "COM" END AS Matriz
,PI * PDI AS PE
,PI_A * PDI_A AS PE_A
,1 AS FACT_A
,'"""+anomesdia3+"""' AS periodo
,CASE WHEN trim(pdt_des_cra)="IC" or trim(pdt_des_cra)="IF" or trim(pdt_des_cra)="EF" or trim(pdt_des_cra)="BF" then 24 
ELSE 23 END AS Negocio_Id
,CASE WHEN Calif_Deudor_Permitida=1 then "AVL" ELSE "" END AS tipo
,CASE WHEN Calif_Deudor_Permitida=1 then Deuda_TotalIFRS ELSE 0 END AS Monto_Avalado
,PROV_d00 as PROV_cm
from """+platinum_temp_dbX+""".Tmp_Fundir_Prov_d00
)
"""

In [0]:
sql_safe(create_tmp_008_Tmp_Prov_Gr_Com_Fact_B1_Result_0719)

sql_safe: query ->  CREATE TABLE IF NOT EXISTS dsr_plt_bcitemp_db.Tmp_Prov_Gr_Com_Fact_B1_Result_0719 
USING DELTA
PARTITIONED BY (periodo)
LOCATION 'abfss://platinum@bcirg2dlssbx.dfs.core.windows.net/plt_normativo_db/prv_hip/Tmp_Prov_Gr_Com_Fact_B1_Result_0719'
AS (
select 
Cod_niid AS ope_num
--,dpf_fec_proceso
,cli_idc as rut
,cli_vrt as dv
--,cli_rzn_soc
,ddr_idc as rut_deudor_fact
,ddr_vrt as dv_deudor_fact
--,ddr_rzn_soc
--,pdt_des_cra
,doc_num_documento
--,doc_num_cuota
,Deuda_TotalIFRS
--,dod_cod_cobranza
--,cod_aec
--,dpf_ind_responsabilidad
--,dpf_fec_vencimiento
,Clasificacion as seg_n2
--,dod_ind_cartera
--,Clasificacion_Cliente
,Clasificacion_Deudor as Clasificacion_Deudor_fact
,Dias_Mora
,CASE WHEN Dias_Mora=0 and Colo_EsCardetIrr=0 then "0"
WHEN Dias_Mora>=1 and Dias_Mora<=29 and Colo_EsCardetIrr=0 then "1-29"
WHEN Dias_Mora>=30 and Dias_Mora<=59 and Colo_EsCardetIrr=0 then "30-59"
WHEN Dias_Mora>=60 and Dias_Mora<=89 and Colo_EsCardetIrr=0 then "60-89"
else "Incumpl" EN

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

## Mensaje Final

In [0]:
dbutils.notebook.exit("{\"coderror\":\"0\", \"msgerror\":\"Notebook termina ejecucion satisfactoriamente\"}")